In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
q = """
with playoffs_21_22 as
    (select '2021-22' as season, left(right(event_name,4),2) as round, event_name,
            CASE
                when datediff('days', date(add_date),date(event_date)) >= 0 then datediff('days', date(add_date),date(event_date))
                else 0
            end as days_out, ticket_type,
         sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_historical_all_1718_2223
    where season = '2021-22' and event_name like '%PO%' and ticket_type in ('Singles', 'Nightly Suites')
    group by event_name, event_date, days_out, ticket_type),
playoffs_21_22_total as
    (select '2021-22' as season, left(right(event_name,4),2) as round, event_name,
            CASE
                when datediff('days', date(add_date),date(event_date)) >= 0 then datediff('days', date(add_date),date(event_date))
                else 0
            end as days_out, 'total' as ticket_type,
         sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_historical_all_1718_2223
    where season = '2021-22' and event_name like '%PO%' and ticket_type in ('Singles', 'Nightly Suites')
    group by event_name, event_date, days_out),
playoffs_22_23 as
    (select '2022-23' as season, left(right(event_name,4),2) as round, event_name,
            CASE
                when datediff('days', date(add_datetime),date(event_date)) >= 0 then datediff('days', date(add_datetime),date(event_date))
                else 0
            end as days_out, ticket_type,
       sum(block_purchase_price) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_ticket_expanded_all_playoffs_2223
    where event_name in ('23POR1G1','23POR1G2','23POR1G3','23POR2G1','23POR2G2','23POR3G1','23POR3G2','23POR4G1','23POR4G2')
        and ticket_type in ('Singles', 'Nightly Suites')
    group by event_name, event_date, days_out, ticket_type),
playoffs_22_23_total as
    (select '2022-23' as season, left(right(event_name,4),2) as round, event_name,
        CASE
            when datediff('days', date(add_datetime),date(event_date)) >= 0 then datediff('days', date(add_datetime),date(event_date))
            else 0
        end as days_out, 'total' as ticket_type,
       sum(block_purchase_price) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_ticket_expanded_all_playoffs_2223
    where event_name in ('23POR1G1','23POR1G2','23POR1G3','23POR2G1','23POR2G2','23POR3G1','23POR3G2','23POR4G1','23POR4G2')
        and ticket_type in ('Singles', 'Nightly Suites')
    group by event_name, event_date, days_out),
playoffs_23_24 as
    (select '2023-24' as season, right(left(product_description,6),2) as round, left(product_description,8) as event_name,
        case
           when round like 'R1' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R2' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           else 0
        end as days_out, ticket_type_playoffs, sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_v_ticket_2324_playoffs
    where ticket_type_playoffs in ('Singles', 'Nightly Suites')
        and event_name != '23-24 Pl'
    group by product_description, days_out, ticket_type_playoffs),
playoffs_23_24_total as
    (select '2023-24' as season, right(left(product_description,6),2) as round, left(product_description,8) as event_name,
        case
           when round like 'R1' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           when round like 'R2' and datediff('days', date(transaction_date),date(event_datetime)) >= 0 then datediff('days', date(transaction_date),date(event_datetime))
           else 0
        end as days_out, 'total' as ticket_type, sum(gross_revenue) as gross_revenue, sum(paid_seats) as paid_seats
    from custom.cth_v_ticket_2324_playoffs
    where ticket_type_playoffs in ('Singles', 'Nightly Suites')
        and event_name != '23-24 Pl'
    group by product_description, days_out)
select *
from playoffs_21_22
union all
select *
from playoffs_22_23
union all
select *
from playoffs_23_24
union all
select *
from playoffs_21_22_total
union all
select *
from playoffs_22_23_total
union all
select *
from playoffs_23_24_total
order by season, round, event_name, days_out desc
"""
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [4]:
df['min_days_out'] = df.apply(lambda row: df[(df['event_name'] == row['event_name'])&(df['ticket_type'] == row['ticket_type'])]['days_out'].min(), axis = 1)
cumdf = df.groupby(by = ['round','event_name', 'ticket_type'], axis = 0)[['gross_revenue','paid_seats']].cumsum().rename(columns = {'gross_revenue':'cum_gross_rev', 'paid_seats':'cum_num_seats'})
dfdf = pd.concat([df,cumdf], axis = 1)
dfdf['final_seats'] = dfdf.apply(lambda row: dfdf[(dfdf['event_name'] == row['event_name'])&(dfdf['ticket_type'] == row['ticket_type'])&
                                                  (dfdf['days_out'] == row['min_days_out'])]['cum_num_seats'].item(), axis = 1)
dfdf['per_seats_in'] = [x/y for x,y in zip(dfdf['cum_num_seats'],dfdf['final_seats'])]
dfdf['num_seats_left'] = dfdf.apply(lambda row: row['final_seats'] - row['cum_num_seats']+ row['paid_seats'], axis = 1)
dfdf = dfdf[['season','round', 'event_name', 'ticket_type','days_out','gross_revenue','paid_seats','num_seats_left','per_seats_in']]
dfdf

,season,round,event_name,ticket_type,days_out,gross_revenue,paid_seats,num_seats_left,per_seats_in
0,2021-22,R1,22POR1G1,Singles,29,154278.00,2165,4924,0.439683
1,2021-22,R1,22POR1G1,total,29,154278.00,2165,5348,0.404824
2,2021-22,R1,22POR1G1,total,28,10349.00,160,3183,0.434742
3,2021-22,R1,22POR1G1,Singles,28,10349.00,160,2759,0.472177
4,2021-22,R1,22POR1G1,total,27,6703.00,105,3023,0.454375
...,...,...,...,...,...,...,...,...,...
792,2023-24,R2,24POR2G4,total,13,21841.91,79,113,0.965021
793,2023-24,R2,24POR2G4,Singles,12,6239.17,29,34,0.994206
794,2023-24,R2,24POR2G4,total,12,6239.17,29,34,0.994856
795,2023-24,R2,24POR2G4,total,11,1267.96,5,5,1.000000


In [5]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = dfdf, table_name= "cth_historical_tier_velocity_playoffs")

In [3]:
df = pd.read_csv("C:\\Users\\riffere\\Desktop\\polygon_coordinates.csv")
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "cth_xy_coordinates_pricelevels")

In [8]:
df = pd.read_csv("C:\\Users\\riffere\\Desktop\\shapes (4).csv")
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "cth_xy_coordinates_suites_seatmap")